In [ ]:
# pip install --quiet --upgrade langchain-text-splitters langchain-community langgraph
# pip install -qU "langchain[aws]"
# pip install python-dotenv
# pip install -qU langchain-aws
# pip install -qU langchain-core

# Investigate:
# pip install unstructured
# pip install "unstructured[pdf]"

### Set Up Credentials and Envitoment Vars

In [3]:
import os
from dotenv import load_dotenv
load_dotenv()

os.environ['AWS_ACCESS_KEY_ID'] = os.getenv('AWS_ACCESS_KEY_ID')
os.environ['AWS_SECRET_ACCESS_KEY'] = os.getenv('AWS_SECRET_ACCESS_KEY')
os.environ['AWS_REGION'] = os.getenv('AWS_REGION')




### Initilaize Gen LLM

In [4]:
from langchain.chat_models import init_chat_model

llm = init_chat_model("anthropic.claude-3-5-sonnet-20240620-v1:0", model_provider="bedrock_converse")

### Initialize Embed LLM

In [ ]:
from langchain_aws import BedrockEmbeddings

embeddings = BedrockEmbeddings(model_id="amazon.titan-embed-text-v2:0")


### In Memory Vector Store

from langchain_core.vectorstores import InMemoryVectorStore

vector_store = InMemoryVectorStore(embeddings)



### Loading Documents from a Directory :)

In [ ]:
from langchain_community.document_loaders import DirectoryLoader

loader = DirectoryLoader("data", glob="*.pdf")
docs = loader.load()
len(docs)

/Users/ktejwani/Local_Ollama_RAG/venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Matplotlib is building the font cache; this may take a moment.
[nltk_data] Error loading averaged_perceptron_tagger_eng: <urlopen
[nltk_data]     error [SSL: CERTIFICATE_VERIFY_FAILED] certificate
[nltk_data]     verify failed: unable to get local issuer certificate
[nltk_data]     (_ssl.c:1028)>
[nltk_data] Error loading punkt_tab: <urlopen error [SSL:
[nltk_data]     CERTIFICATE_VERIFY_FAILED] certificate verify failed:
[nltk_data]     unable to get local issuer certificate (_ssl.c:1028)>
Data-loss while decompressing corrupted data
Data-loss while decompressing corrupted data
Data-loss while decompressing corrupted data
Data-loss while decompressing corrupted data
Data-loss while decompressing corrupted data
Data-lo

PDFInfoNotInstalledError: Unable to get page count. Is poppler installed and in PATH?